# 🇫🇷 Fine-tuning Vigostral-7B-Chat sur votre style personnel

Ce notebook vous guide pour fine-tuner le modèle conversationnel français **Vigostral-7B-Chat** sur vos dialogues personnels, en utilisant **LoRA** (Low-Rank Adaptation) pour s'adapter aux contraintes mémoire du GPU T4 gratuit de Google Colab.

## 📋 Prérequis

1. **GPU T4 activé** : Runtime → Change runtime type → T4 GPU
2. **Dataset de dialogues** au format JSONL
3. **~30 minutes** de votre temps

## 🎯 Objectif

Adapter Vigostral-7B-Chat (modèle conversationnel français pré-entraîné sur 213k dialogues) à votre style d'écriture personnel en fine-tunant sur vos propres conversations.

## ⚙️ Technique : LoRA (Low-Rank Adaptation)

- **Problème** : Fine-tuner 7B paramètres nécessite ~50GB de VRAM (impossible sur T4)
- **Solution** : LoRA ne fine-tune que ~1% des paramètres (~70M)
- **Avantage** : Tient dans 16GB, training rapide, qualité préservée

---

## ⚠️ IMPORTANT : Vérifiez le GPU

**Avant de commencer**, assurez-vous que le GPU T4 est activé :
1. En haut à droite : **Runtime** → **Change runtime type**
2. Sélectionnez **T4 GPU**
3. Cliquez **Save**

In [ ]:
# Vérifier que le GPU est disponible
import torch

if torch.cuda.is_available():
    print(f"✅ GPU détecté : {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire totale : {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ ERREUR : Aucun GPU détecté !")
    print("   → Allez dans Runtime → Change runtime type → Sélectionnez T4 GPU")
    raise RuntimeError("GPU requis pour ce notebook")

---

## 📦 Installation des dépendances

Installation de toutes les bibliothèques nécessaires. **Durée : 2-3 minutes**

⚠️ **IMPORTANT** : Après l'installation, vous DEVEZ redémarrer le runtime avant de continuer !

In [ ]:
print("📦 Installation des bibliothèques nécessaires...")
print("   (Cela peut prendre 2-3 minutes)\n")

# Installer chaque package séparément pour plus de robustesse
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U trl

# ⚠️ IMPORTANT : Forcer pyarrow à une version compatible avec cudf-cu12 de Colab
print("📌 Installation de pyarrow compatible avec Colab...")
!pip install -q 'pyarrow>=14.0.0,<20.0.0'

# Installer datasets 3.x (compatible avec pyarrow < 20.0)
print("📌 Installation de datasets compatible avec pyarrow < 20.0...")
!pip install -q 'datasets>=3.0.0,<4.0.0'

print("\n✅ Installation terminée !\n")
print("⚠️  ATTENTION : Vous DEVEZ redémarrer le runtime maintenant !")
print("   → Runtime → Restart runtime (ou Ctrl+M .)")
print("   → Puis re-exécutez la cellule de VÉRIFICATION (ne pas réexécuter cette cellule d'installation)\n")
print("💡 Raison : bitsandbytes nécessite un restart pour charger les extensions CUDA")

---

## ✅ Vérification des bibliothèques (APRÈS RESTART)

**Exécutez cette cellule UNIQUEMENT après avoir redémarré le runtime.**

Cette cellule vérifie que toutes les bibliothèques sont correctement installées et que bitsandbytes peut charger ses extensions CUDA.

In [ ]:
print("🔍 Vérification des bibliothèques...\n")

# Vérifier les imports
try:
    import torch
    print(f"✅ torch: {torch.__version__}")
except Exception as e:
    print(f"❌ torch: {e}")
    raise

try:
    import transformers
    print(f"✅ transformers: {transformers.__version__}")
except Exception as e:
    print(f"❌ transformers: {e}")
    raise

try:
    import peft
    print(f"✅ peft: {peft.__version__}")
except Exception as e:
    print(f"❌ peft: {e}")
    raise

try:
    import trl
    print(f"✅ trl: {trl.__version__}")
except Exception as e:
    print(f"❌ trl: {e}")
    raise

try:
    import bitsandbytes
    print(f"✅ bitsandbytes: {bitsandbytes.__version__}")
except Exception as e:
    print(f"❌ bitsandbytes: {e}")
    raise

try:
    import datasets
    print(f"✅ datasets: {datasets.__version__}")
except Exception as e:
    print(f"❌ datasets: {e}")
    raise

try:
    import pyarrow
    print(f"✅ pyarrow: {pyarrow.__version__}")
    if int(pyarrow.__version__.split('.')[0]) >= 20:
        print("   ⚠️  ATTENTION : pyarrow >= 20 peut causer des problèmes")
except Exception as e:
    print(f"❌ pyarrow: {e}")

# Test crucial : vérifier que bitsandbytes peut charger ses extensions CUDA
print("\n🔍 Test de bitsandbytes avec CUDA...")
try:
    import bitsandbytes as bnb
    # Tenter de créer un optimizer (cela force le chargement des extensions CUDA)
    test_param = torch.nn.Parameter(torch.randn(10, 10).cuda())
    optimizer = bnb.optim.Adam8bit([test_param], lr=1e-3)
    print("✅ bitsandbytes fonctionne correctement avec CUDA !")
    del optimizer, test_param
except Exception as e:
    print(f"❌ ERREUR bitsandbytes : {e}")
    print("\n⚠️  Vous devez redémarrer le runtime !")
    print("   → Runtime → Restart runtime")
    raise RuntimeError("bitsandbytes nécessite un restart du runtime")

print("\n✅ Toutes les vérifications sont passées !")
print("💡 Vous pouvez maintenant continuer avec le reste du notebook.")

---

## 📤 Upload de votre dataset

Uploadez votre fichier `combined_dataset.jsonl` (format : une ligne par dialogue)

In [ ]:
from google.colab import files
import shutil

print("📤 Uploadez votre fichier combined_dataset.jsonl")
uploaded = files.upload()

# Trouver le fichier uploadé (peut être combined_dataset.jsonl ou combined_dataset (2).jsonl, etc.)
dataset_file = None
for filename in uploaded.keys():
    if 'combined_dataset' in filename and filename.endswith('.jsonl'):
        dataset_file = filename
        break

if dataset_file is None:
    raise FileNotFoundError("Aucun fichier combined_dataset.jsonl trouvé. Réessayez l'upload.")

# Si le fichier n'est pas exactement "combined_dataset.jsonl", le renommer
if dataset_file != 'combined_dataset.jsonl':
    print(f"📝 Renommage de '{dataset_file}' en 'combined_dataset.jsonl'")
    shutil.move(dataset_file, 'combined_dataset.jsonl')

print("✅ Dataset uploadé avec succès !")

---

## 📊 Chargement et validation du dataset

In [ ]:
import json

# Charger le dataset
dialogues = []
with open('combined_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        dialogues.append(json.loads(line))

print(f"✅ {len(dialogues)} dialogues chargés")

# Afficher un exemple
print("\n📋 Exemple de dialogue :")
print(json.dumps(dialogues[0], indent=2, ensure_ascii=False))

---

## 🔄 Conversion au format Vigostral

Vigostral utilise le format Llama-2 : `<s>[INST] question [/INST] réponse </s>`

In [ ]:
def format_dialogue_for_vigostral(dialogue):
    """
    Convertit un dialogue du format OpenAI au format Vigostral.
    
    Vigostral utilise le format Llama-2 :
    <s>[INST] question [/INST] réponse </s>
    """
    messages = dialogue['messages']
    formatted_text = "<s>"
    
    for i in range(0, len(messages), 2):
        if i < len(messages) and messages[i]['role'] == 'user':
            user_msg = messages[i]['content']
            formatted_text += f"[INST] {user_msg} [/INST]"
        
        if i + 1 < len(messages) and messages[i + 1]['role'] == 'assistant':
            assistant_msg = messages[i + 1]['content']
            formatted_text += f" {assistant_msg} </s>"
    
    return formatted_text

# Convertir tous les dialogues
formatted_dialogues = [format_dialogue_for_vigostral(d) for d in dialogues]

print(f"✅ {len(formatted_dialogues)} dialogues formatés")
print("\n📋 Exemple de dialogue formaté :")
print(formatted_dialogues[0][:500] + "...")

---

## 🤖 Chargement du modèle Vigostral-7B-Chat

On charge le modèle avec :
- **Quantization 4-bit** : Réduit la taille de ~14GB à ~4GB
- **LoRA** : Ajoute des adaptateurs entraînables (~70M params)

**⏳ Durée estimée : 5-10 minutes**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ✅ Nom correct du modèle
model_name = "bofenghuang/vigostral-7b-chat"

# Configuration de la quantization 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("📥 Téléchargement du modèle Vigostral-7B-Chat...")
print("   (Peut prendre 5-10 minutes la première fois)\n")

# Charger le modèle en 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Modèle chargé avec succès !")
print(f"   Taille du modèle : {model.get_memory_footprint() / 1e9:.2f} GB")

---

## 🎛️ Configuration LoRA

LoRA ajoute de petites matrices entraînables aux couches d'attention du modèle.

**Paramètres** :
- `r=16` : Rang des matrices (plus haut = plus de capacité, mais plus de mémoire)
- `lora_alpha=32` : Facteur de scaling
- `target_modules` : Couches à adapter (query et value projections)

In [ ]:
# Préparer le modèle pour LoRA
model = prepare_model_for_kbit_training(model)

# Configuration LoRA
lora_config = LoraConfig(
    r=16,  # Rang des matrices LoRA
    lora_alpha=32,  # Facteur de scaling
    target_modules=["q_proj", "v_proj"],  # Couches à adapter
    lora_dropout=0.05,  # Dropout pour régularisation
    bias="none",
    task_type="CAUSAL_LM",
)

# Appliquer LoRA au modèle
model = get_peft_model(model, lora_config)

# Afficher le nombre de paramètres entraînables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA activé !")
print(f"   Paramètres entraînables : {trainable_params:,} ({100 * trainable_params / all_params:.2f}%)")
print(f"   Paramètres totaux : {all_params:,}")

---

## 📚 Préparation du dataset pour l'entraînement

In [ ]:
from datasets import Dataset

# Créer un dataset HuggingFace
dataset_dict = {"text": formatted_dialogues}
dataset = Dataset.from_dict(dataset_dict)

# Split train/validation (90/10)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"✅ Dataset préparé :")
print(f"   Train : {len(dataset['train'])} dialogues")
print(f"   Validation : {len(dataset['test'])} dialogues")

---

## 🚀 Configuration de l'entraînement

**Paramètres optimisés pour T4** :
- `num_train_epochs=3` : 3 passages sur le dataset
- `per_device_train_batch_size=1` : 1 exemple à la fois (limite mémoire)
- `gradient_accumulation_steps=4` : Accumule 4 gradients avant mise à jour (batch effectif = 4)
- `learning_rate=2e-4` : Learning rate pour LoRA
- `save_strategy="steps"` : Sauvegarde tous les 50 steps (évite de perdre le travail en cas de crash)
- `fp16=True` : Utilise float16 pour économiser la mémoire

In [ ]:
from trl import SFTTrainer, SFTConfig

# ✅ Configuration avec SFTConfig (pas TrainingArguments)
sft_config = SFTConfig(
    output_dir="./vigostral-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="steps",      # ✅ Sauvegarder tous les N steps
    save_steps=50,               # ✅ Tous les 50 steps (~3-5 min)
    logging_steps=10,
    optim="paged_adamw_8bit",    # Optimizer optimisé pour mémoire
    warmup_steps=50,
    report_to="none",            # Désactiver wandb/tensorboard
    dataset_text_field="text",   # ✅ Champ contenant le texte
    packing=False,               # ✅ Désactiver le packing
    max_length=512,              # ✅ Longueur max (pas max_seq_length !)
)

# Créer le trainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

print("✅ Trainer configuré !")
print(f"   Nombre total de steps : ~{len(dataset['train']) * 3 // 4}")
print(f"   Checkpoints seront sauvegardés tous les 50 steps")

---

## 🎯 Lancement de l'entraînement

**⏳ Durée estimée : 20-30 minutes sur T4 GPU**

⚠️ **IMPORTANT** : Gardez cet onglet ouvert et visible pendant l'entraînement pour éviter la déconnexion !

In [ ]:
import time

print("🚀 Début de l'entraînement...\n")
print("⚠️  Gardez cet onglet ouvert pendant le training !\n")
start_time = time.time()

# Lancer le training
trainer.train()

elapsed_time = time.time() - start_time
print(f"\n✅ Entraînement terminé en {elapsed_time / 60:.1f} minutes !")

---

## 💾 Sauvegarde du modèle fine-tuné

In [ ]:
# Sauvegarder le modèle et les adaptateurs LoRA
model.save_pretrained("./vigostral-finetuned-final")
tokenizer.save_pretrained("./vigostral-finetuned-final")

print("✅ Modèle sauvegardé dans ./vigostral-finetuned-final")

---

## 🧪 Test du modèle fine-tuné

Testons le modèle avec quelques prompts pour voir comment il répond dans votre style.

In [ ]:
def chat(prompt, max_length=200, temperature=0.7, top_p=0.9):
    """
    Génère une réponse du modèle fine-tuné.
    """
    # Formater le prompt au format Vigostral
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    
    # Tokenizer
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    # Générer
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Décoder la réponse
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraire seulement la réponse (après [/INST])
    if "[/INST]" in response:
        response = response.split("[/INST]")[1].strip()
    
    return response

# Test avec quelques prompts
test_prompts = [
    "Explique-moi ton projet principal",
    "Qu'est-ce que tu penses du machine learning ?",
    "Comment tu travailles au quotidien ?",
]

print("\n🧪 Test du modèle fine-tuné :\n")
print("=" * 80)

for prompt in test_prompts:
    print(f"\n👤 User: {prompt}")
    response = chat(prompt)
    print(f"🤖 Assistant: {response}")
    print("-" * 80)

---

## 💬 Mode interactif

Testez librement votre modèle !

In [ ]:
print("💬 Mode chat interactif")
print("Tapez 'quit' pour quitter\n")

while True:
    user_input = input("👤 Vous: ")
    
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("👋 Au revoir !")
        break
    
    if not user_input.strip():
        continue
    
    response = chat(user_input)
    print(f"🤖 Assistant: {response}\n")

---

## 📦 Télécharger le modèle fine-tuné

Pour utiliser votre modèle localement, téléchargez les fichiers sauvegardés.

In [ ]:
# Créer une archive zip du modèle
!zip -r vigostral-finetuned-final.zip vigostral-finetuned-final/

# Télécharger l'archive
from google.colab import files
files.download('vigostral-finetuned-final.zip')

print("✅ Modèle téléchargé ! Taille des adaptateurs LoRA : ~100-200 MB")

---

## 🎉 Félicitations !

Vous avez fine-tuné Vigostral-7B-Chat sur votre style personnel !

### 📝 Prochaines étapes

1. **Tester localement** :
   ```bash
   # Dézippez le modèle
   unzip vigostral-finetuned-final.zip
   
   # Lancez l'interface Gradio
   python chat_gradio.py
   ```

2. **Améliorer le modèle** :
   - Ajouter plus de dialogues (200+ recommandé)
   - Augmenter les epochs (`num_train_epochs=5`)
   - Ajuster `r` dans LoRA (16 → 32 pour plus de capacité)

3. **Intégrer avec nanochat** : Voir `INTEGRATION_NANOCHAT.md`

---

**Questions ?** Ouvrez une issue sur GitHub : https://github.com/cladjidane/nanochat-french-tutorial/issues

**Made with ❤️ for the French AI community**